# Question 5.a

### (i) Exact test size of the LRT test

In [1]:
n <- 80
p <- c(1/6, 1/6, 1/6, 3/10, 2/10)

In [2]:
# generate all integer combinations t0, t1, t2, t3, t4 such that t0 + t1 + t2 + t3 + t4 = n
grid <- expand.grid(t0 = 0:n, t1 = 0:n, t2 = 0:n, t3 = 0:n)
grid <- subset(grid, t0 + t1 + t2 + t3 <= n) # filter combinations that sum maximum to n
grid$t4 <- n - (grid$t0 + grid$t1 + grid$t2 + grid$t3) # add t4 to the grid
# grid$t4 # technically, there are no t4 negatives values


In [3]:
# compute the statistic Lambda for each combination
log_ratio <- function(t, p, n) {
  terms <- ifelse(t > 0, t * log(t / (n * p)), 0) 
  return(2 * sum(terms))
}

lambda <- apply(grid, 1, function(t) log_ratio(t, p, n))

In [36]:
# compute the probability of each combination using multinomial distribution
prob <- function(t, n) {
  return(dmultinom(t, size = n, prob = p))
}

probs <- apply(grid, 1, function(t) prob(t, n))

In [37]:
# chi squared quantile
alpha <- 0.05
q <- qchisq(1 - alpha, df = length(p) - 1) # 4 degrees of freedom

In [ ]:
# compute the exact size of the test
size <- sum(probs[lambda > q]) # sum of probabilities for lambda > q
cat("Exact size of the test:", size, "\n")

Exact size of the test: 0.05274593 


### (ii) Exact test size of the Pearson test

In [ ]:
n <- 80
p <- c(1/6, 1/6, 1/6, 3/10, 2/10)

# generate the grid
grid <- expand.grid(t0 = 0:n, t1 = 0:n, t2 = 0:n, t3 = 0:n)
grid <- subset(grid, t0 + t1 + t2 + t3 <= n) # filter combinations that sum maximum to n
grid$t4 <- n - (grid$t0 + grid$t1 + grid$t2 + grid$t3) # add t4 to the grid

grid_matrix <- as.matrix(grid) # matrix to speed up calculations

# compute Pearson statistic for each row
pearson <- function(t, p, n) {
  expected <- n * p
  sum((t - expected)^2 / expected)
}
Q <- apply(grid_matrix, 1, function(t) pearson(t, p, n))

# compute the multinomial probabilities under H0
probs <- apply(grid_matrix, 1, function(t) dmultinom(t, prob = p))

# compute the exact size
alpha_pearson <- sum(probs[Q > q])

In [58]:
cat("Exact size of the test using Pearson's statistic:", alpha_pearson, "\n")

Exact size of the test using Pearson's statistic: 0.0489614 


## # Question 5.b
### (ii) Proportion of rejections for the LRT test

In [4]:
p2 <- 0.1
p1 <- 4*p2
p0 <- 0.15
p3 <- 0.25
p4 <- 1 - p0 - p1 - p2 - p3 # 0.1
p <- c(p0, p1, p2, p3, p4)
cat(p)

0.15 0.4 0.1 0.25 0.1

In [ ]:
N = 13000 # number of simulations
n <- 45 # sample size
alpha <- 0.05 
q <- qchisq(1 - alpha, df = 1) # ~3.84

log_ratio <- function(t, p, n) {
  terms <- ifelse(t > 0, t * log(t / (n * p)), 0) 
  return(2 * sum(terms))
}

rejections <- 0 # initialization

for (i in 1:N) {
  # generate the data
  t <- rmultinom(1, n, p)

  # compute the statistic
  lambda <- log_ratio(t, p, n)

  # compute the p-value
  if (lambda > q) {
    rejections <- rejections + 1 # increment rejection count
  }
}

In [9]:
cat("Rejection rate:", rejections / N, "\n")

Rejection rate: 0.4454615 
